In [ ]:
from src.v1.inputresolver import eqvars, default_out, getallvars, var_matched_cons, invert_edges
from src.v1.presolver import var_matched_cons
from graph.graphview import bipartite_repr
from src.v1.symbolic import Var
import gurobipy as gp
from gurobipy import GRB

### Test equations with same output

In [3]:
a,b,c,d = Var('a'), Var('b'), Var('c'), Var('d')
eqlist = [(b,c+1),(a,b-2),(a,2*d+3)]
eqs = {idx: eq for idx, eq in enumerate(eqlist)}
dout = default_out(eqs)
vrs = getallvars(eqs)
eqns = eqs.keys()
eqv = eqvars(eqs)
G, edges_original = bipartite_repr(eqv)
edges = invert_edges(edges_original)

In [4]:
not_input = []
n_eqs = len(eqns)

In [5]:
m = gp.Model('minrevassign')
m.setParam('OutputFlag', False)
x = m.addVars(edges, name="assign", vtype=GRB.BINARY)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2527862
Academic license 2527862 - for non-commercial use only - registered to no___@mit.edu
Set parameter WLSSecret
Set parameter LicenseID to value 2527862
Academic license 2527862 - for non-commercial use only - registered to no___@mit.edu


In [6]:
# Matching eqs:
fcons = m.addConstrs((x.sum(j,'*') == 1 for j in eqns), name='equations')
varcons = m.addConstrs((var_matched_cons(x, j, not_input) for j in vrs), name='variables')
m.setObjective(gp.quicksum([x[(key, var)] for key,var in dout.items()]), GRB.MAXIMIZE)
# Elimination
#m.addConstr(gp.quicksum(x[(r,j)] for r,j in sol)<= n_eqs-1) 

In [7]:
m.optimize()

In [8]:
m.objVal

2.0

In [9]:
sol = [(r,j) for (r, j) in edges if x[r,j].x>1e-6]

In [10]:
sol_outset = {key:val for key,val in sol}

In [11]:
sol_outset

{0: b, 1: a, 2: d}